In [ ]:
# check the package version
from importlib.metadata import version

pkgs = ["matplotlib", "numpy", "tiktoken", "torch"]
for p in pkgs:
    print(f"{p} version: {version(p)}")



matplotlib version: 3.10.3
numpy version: 2.3.1
tiktoken version: 0.9.0
torch version: 2.7.1


## 使用 GPT 生成文本
- 上一章中的代码初始化 GPT 模型
  - 上面使用0.1的dropout，但现在的llm训练中通常没有dropout
  - 现在的llm也不会在查询，键和值矩阵的nn.Linear层中使用偏差向量 (与早期的GPT模型不同)，这是通过设置“qkv_bias”: False来实现的

## 计算文本生成损失：交叉熵和困惑度
- 假设我们有一个inputs张量，包含了2个训练样本（行）的标记ID。
- 对应于inputs，targets包含了我们希望模型生成的期望标记ID。
- 请注意，targets是inputs向右移动了一个位置，正如第2章中实现数据加载器时所解释的那样
- 将inputs输入模型后，我们获得了包含3个标记的2个输入样本的logits向量。
- 每个标记都是一个50,257维的向量，对应于词汇表的大小。
- 应用softmax函数，我们可以将logits张量转换为一个相同维度的张量，其中包含概率分数。

In [ ]:
import torch
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [588,  428,  11311]]) #  " really like chocolate"]

batch_idx = 0
target_probas_1 = probas[batch_idx, [0, 1, 2], targets[batch_idx]]
print("Batch 1:", target_probas_1)

batch_idx = 1
target_probas_2 = probas[1, [0, 1, 2], targets[1]]
print("Batch 2:", target_probas_2)



Batch 1: tensor([7.4540e-05, 3.1061e-05, 1.1563e-05])
Batch 2: tensor([3.9836e-05, 1.6783e-05, 4.7559e-06])
- 我们希望最大化所有这些值，使它们接近1的概率。
- 在数学优化中，最大化概率分数的对数比分数值本身更容易

In [ ]:

# 计算所有标记的预测概率的对数值
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)
# 接下来，我们计算平均对数概率
# 对所有标记的概率对数值求均值
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)
# tensor(-10.7722)

- 目标是通过优化模型权重，使得这个平均对数概率尽可能大。
- 由于对数函数的特性，最大可能的值是0，而我们目前远离0。
- 在深度学习中，我们通常不是最大化平均对数概率，而是遵循标准惯例来最小化平均对数概率的负值；在我们的例子中，不是最大化-10.7722使其接近0，在深度学习中，我们会最小化10.7722使其接近0。
- 负-10.7722的值，即10.7722，在深度学习中也被称为交叉熵损失。
- PyTorch 已经实现了一个 cross_entropy 函数，该函数执行了前面的步骤。

In [ ]:

neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)
# tensor(10.7722)

# Logits向量的形状 (batch_size, num_tokens, vocab_size)
print("Logits shape:", logits.shape)

# 目标向量的形状 (batch_size, num_tokens)
print("Targets shape:", targets.shape)
# Logits shape: torch.Size([2, 3, 50257])
# Targets shape: torch.Size([2, 3])

# 对于PyTorch中的entropy_loss函数，我们希望通过在批次(batch)维度上合并它们来展平(flatten)这些张量：
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)
# Flattened logits: torch.Size([6, 50257])
# Flattened targets: torch.Size([6])

# 请注意，目标（targets）是标记ID，它们也代表了我们希望在logits张量中最大化的索引位置。
# PyTorch中的cross_entropy函数会自动地将softmax和对数概率计算应用到这些要最大化标记索引的logits上

loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)
# tensor(10.7722)